## The general program flow in TensorFlow

Usually, a program in TensorFlow is divided into two parts; the construction phase and the execution phase. In the construction phase, the computational graph that TensorFlow uses to perform its calculations are set up. In the execution phase, TensorFlow evaluates any procedure that was defined in the construction phase.
Program flow in TensorFlow - Construction phase

Here, the architecture for the neural network will be set up, along with the cost function and an optimizer class used during training of the network. Note that TensorFlow uses a different convention for the weighting done in each neuron in each layer within the network than in the implementation using Autograd. The matrix-vector multiplication between the input from the previous layer and the weighting at the neuron at current layer in the program using Autograd, is the transpose of the convention used in TensorFlow. But it will not affect that much our construction, as TensorFlow takes care of most of the computations. The only thing we have to be aware of, is how the dimensions are for our inputs.
Program flow in TensorFlow - Execution phase

The computation graph has been defined, and is ready to be evaluated. In order to get access to the graph, it has to be initialized and be runned within a Session.
The full program modeling logistic population growth using TensorFlow 

https://compphysics.github.io/MachineLearning/doc/pub/odenn/html/._odenn-bs021.html

# Does not work with tensorflow 2

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Just to reset the graph such that it is possible to rerun this in a
# Jupyter cell without resetting the whole kernel.
tf.reset_default_graph()

# Set a seed to ensure getting the same results from every run
tf.set_random_seed(4155)

Nt = 10
T = 1
t = np.linspace(0,T, Nt)

## The construction phase

# Convert the values the trial solution is evaluated at to a tensor.
t_tf = tf.convert_to_tensor(t.reshape(-1,1),dtype=tf.float64)
zeros = tf.reshape(tf.convert_to_tensor(np.zeros(t.shape)),shape=(-1,1))

# Define the parameters of the equation
alpha = tf.constant(2.,dtype=tf.float64)
A = tf.constant(1.,dtype=tf.float64)
g0 = tf.constant(1.2,dtype=tf.float64)

num_iter = 100000

# Define the number of neurons at each hidden layer
num_hidden_neurons = [100,50,25]
num_hidden_layers = np.size(num_hidden_neurons)

# Construct the network.
# tf.name_scope is used to group each step in the construction,
# just for a more organized visualization in TensorBoard
with tf.name_scope('dnn'):

    # Input layer
    previous_layer = t_tf

    # Hidden layers
    for l in range(num_hidden_layers):
        current_layer = tf.layers.dense(previous_layer, num_hidden_neurons[l], name='hidden%d'%(l+1), activation=tf.nn.sigmoid)
        previous_layer = current_layer

    # Output layer
    dnn_output = tf.layers.dense(previous_layer, 1, name='output')

# Define the cost function
with tf.name_scope('cost'):
    g_trial = g0 + t_tf*dnn_output
    d_g_trial = tf.gradients(g_trial,t_tf)

    func = alpha*g_trial*(A - g_trial)
    cost = tf.losses.mean_squared_error(zeros, d_g_trial[0] - func)


# Choose the method to minimize the cost function, along with a learning rate
learning_rate = 1e-2
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    traning_op = optimizer.minimize(cost)

# Set up a referance to the result from the neural network:
g_dnn_tf = None

# Define a node that initializes all of the other nodes in the computational graph
# used by TensorFlow:
init = tf.global_variables_initializer()

## Execution phase

# Start a session where the graph defined from the construction phase can be evaluated at:
with tf.Session() as sess:
    # Initialize the whole graph
    init.run()

    # Evaluate the initial cost:
    print('Initial cost: %g'%cost.eval())

    # The training of the network:
    for i in range(num_iter):
        sess.run(traning_op)

        # If one desires to see how the cost function behaves for each iteration:
        #if i % 1000 == 0:
        #    print(cost.eval())

    # Training is done, and we have an approximate solution to the ODE
    print('Final cost: %g'%cost.eval())

    # Store the result
    g_dnn_tf = g_trial.eval()

# Compare with analytical solution
def get_parameters():
    alpha = 2
    A = 1
    g0 = 1.2
    return alpha, A, g0

def g_analytic(t):
    alpha,A, g0 = get_parameters()
    return A*g0/(g0 + (A - g0)*np.exp(-alpha*A*t))

g_analytical = g_analytic(t)
diff_tf = g_dnn_tf - g_analytical.reshape(-1,1)

print('\nMax absolute difference between the analytical solution and solution from TensorFlow DNN: %g'%np.max(np.abs(diff_tf)))

# Plot the result
plt.figure(figsize=(10,10))

plt.title('Numerical solutions of the ODE')

plt.plot(t, g_dnn_tf)
plt.plot(t, g_analytical)

plt.legend(['dnn, tensorflow', 'exact'])
plt.xlabel('Time t')
plt.ylabel('g(t)')

plt.show()

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.